In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import os 

In [ ]:
from torch.utils.data import Dataset

class WaterAccessDataset(Dataset):

    # constructor
    def __init__(self, csv_path, image_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform

    # len(dataset)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        # get single row in dataframe and convert score to tensor
        row = self.data.iloc[index]
        tile_id = row['tile_id']
        label = torch.tensor(row['score'], dtype=torch.float32)

        # load and process image
        img_path = os.path.join(self.image_dir, f"sentinel2_{tile_id}.png")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # remove non feature columns
        tab = row.drop(['tile_id', 'score']).values.astype('float32')
        
        tab = torch.tensor(tab)    

        return (image, tab), label

In [ ]:
# --- image transformations ---

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224,0.225])
])  

In [ ]:
dataset = WaterAccessDataset(csv_path='data/tile_features_scaled.csv', 
                             image_dir='earth_engine/converted_png', 
                             transform = transform)

In [ ]:
# --- test sample ---

sample = dataset[0]
(image, tabular), label = sample

print("image shape:", image.shape) 
print("tabular shape:", tabular.shape)  
print("label (the score):", label)

In [ ]:
# --- CNN super silly tabular fusion model 3000 ---

class CNNTFMModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()

        resnet = models.resnet18(pretrained=True)

        # takes all layers except final classification layer 
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])
        self.cnn_out_dim = resnet.fc.in_features

        self.fc = nn.Sequential(

            # number of imputs = image features + tabular features
            nn.Linear(self.cnn_out_dim + tabular_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

    # this is automatically called - outputs the predicted score
    def forward(self, image, tabular):

        # processes image
        cnn_feat = self.cnn(image)
        cnn_feat = cnn_feat.view(image.size(0), -1)

        # concatenates image feats and tab feats 
        x = torch.cat((cnn_feat, tabular), dim=1)

        # passes x through the layers and output score
        return self.fc(x).squeeze()

In [ ]:
# --- create the model --- 
tabular_dim = dataset[0][0][1].shape[0]  # length of the feature vector
model = CNNTFMModel(tabular_dim=tabular_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

val_ratio = 0.2
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset) - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"train length: {len(train_dataset)}, validation length: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
epochs = 10

for epoch in range(epochs):
    
    total_loss = 0
    model.train()  # set model to training mode

    for batch_idx, batch in enumerate(train_loader):
        (images, tabular), labels = batch
        images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)

        optimizer.zero_grad()  # reset optimizer gradients 
        predictions = model(images, tabular)
        loss = criterion(predictions, labels)  # compute the loss
        loss.backward()  # compute gradient
        optimizer.step()   # update weights

        total_loss += loss.item()

        print(f"epoch {epoch+1} | batch {batch_idx+1}/{len(train_loader)} | batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.4f}")

    # --- validation phase ---
    model.eval()  # change to evaluation mode
    val_loss = 0

    with torch.no_grad():  # do not need gradients
        for (images, tabular), labels in val_loader:
            images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
            predictions = model(images, tabular)
            loss = criterion(predictions, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"epoch {epoch+1}/{epochs} - val Loss: {avg_val_loss:.4f}")

print("training completed")

torch.save(model.state_dict(), 'model.pth')
print("model saved :)")